In [1]:
import requests
import pandas as pd
import json

In [ ]:

accident = pd.read_csv('Road Accident Data.csv')
sample_size = int(len(accident)* 0.0001)
accident = accident.sample(n=sample_size, random_state=42)

try:
    accident['Accident Date'] = pd.to_datetime(accident['Accident Date'], errors='coerce').dt.strftime('%Y-%m-%d')
except Exception as e:
    print(f"Erreur lors de la conversion des dates : {e}")


In [ ]:
url = "https://archive-api.open-meteo.com/v1/era5"

weather_data = []

for _, row in accident.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    date = row['Accident Date']
    index = row["Accident_Index"]

    # Paramètres pour l'API Open-Meteo
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": date,
        "end_date": date,
        "daily": "temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max,shortwave_radiation_sum,sunrise,sunset",
        "timezone": "auto"
    }

    try:
        response = requests.get(url, params=params)
        data = response.json()

        # Vérification si les données "daily" sont disponibles
        if "daily" in data:
            daily_data = data["daily"]

            # Extraction et calcul des données nécessaires
            max_temp = daily_data["temperature_2m_max"][0] if "temperature_2m_max" in daily_data else None
            min_temp = daily_data["temperature_2m_min"][0] if "temperature_2m_min" in daily_data else None
            total_precipitation = daily_data["precipitation_sum"][0] if "precipitation_sum" in daily_data else None
            wind_speed = daily_data["windspeed_10m_max"][0] if "windspeed_10m_max" in daily_data else None
            solar_radiation = daily_data["shortwave_radiation_sum"][0] if "shortwave_radiation_sum" in daily_data else None

            # Calcul de la durée totale de jour en heures
            sunrise = pd.to_datetime(daily_data["sunrise"][0]) if "sunrise" in daily_data else None
            sunset = pd.to_datetime(daily_data["sunset"][0]) if "sunset" in daily_data else None
            day_length = None
            if sunrise and sunset:
                day_length = (sunset - sunrise).seconds / 3600  # Convertir en heures

            # Ajouter les données collectées dans la liste
            weather_data.append({
                "AccidentIndex": index,
                "Date": date,
                "MaxTemperature": max_temp,
                "MinTemperature": min_temp,
                "TotalPrecipitation": total_precipitation,
                "MaxWindSpeed": wind_speed,
                "SolarRadiation": solar_radiation,
                "DayLengthHours": day_length
            })

           
           
    except Exception as e:
        print(f"Erreur pour l'index {index}: {e}")

# Sauvegarder les données météo dans un fichier JSON
with open('weather_data.json', 'w') as json_file:
    json.dump(weather_data, json_file, indent=4)

print("Données météorologiques ajoutées avec succès !")

Index 2.01E+12 - Sunrise: 2021-06-01 03:43:00, Sunset: 2021-06-01 20:01:00, Day Length: 16.3 hours
Index 200920E008803 - Sunrise: 2021-01-14 08:11:00, Sunset: 2021-01-14 16:21:00, Day Length: 8.166666666666666 hours
Index 200920D177103 - Sunrise: 2021-07-22 04:12:00, Sunset: 2021-07-22 20:14:00, Day Length: 16.033333333333335 hours
Index 2.01E+12 - Sunrise: 2022-02-24 07:06:00, Sunset: 2022-02-24 17:33:00, Day Length: 10.45 hours
Index 2.01E+12 - Sunrise: 2021-05-07 04:28:00, Sunset: 2021-05-07 19:36:00, Day Length: 15.133333333333333 hours
Index 201032D342810 - Sunrise: 2022-11-09 07:15:00, Sunset: 2022-11-09 16:16:00, Day Length: 9.016666666666667 hours
Index 200901VK30104 - Sunrise: 2021-04-16 05:02:00, Sunset: 2021-04-16 18:59:00, Day Length: 13.95 hours
Index 2.01E+12 - Sunrise: 2022-01-12 08:17:00, Sunset: 2022-01-12 16:31:00, Day Length: 8.233333333333333 hours
Index 201060L130221 - Sunrise: 2022-07-17 04:11:00, Sunset: 2022-07-17 20:35:00, Day Length: 16.4 hours
Index 2.01E+12 